In [ ]:
import sys, asyncio
if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import numpy as np, pandas as pd, holoviews as hv, panel as pn
from holoviews import opts, dim
hv.extension('bokeh'); pn.extension('bokeh')

In [ ]:
def chord_by_network(fc, network_labels, TITLE, topk_abs=100, min_abs=0.1):
    fc = np.asarray(fc, float)
    labs = np.asarray(network_labels)
    iu = np.triu_indices_from(fc, 1)
    df = pd.DataFrame({'i': iu[0], 'j': iu[1], 'w': fc[iu]})
    df['src'] = labs[df['i']]; df['tgt'] = labs[df['j']]
    df['absw'] = df['w'].abs()
    df = df[df['absw'] >= float(min_abs)]
    links = (df.groupby(['src','tgt'], as_index=False)['absw']
               .mean().sort_values('absw', ascending=False)
               .head(int(topk_abs))
               .rename(columns={'src':'source','tgt':'target','absw':'value'})
               [['source','target','value']])
    nodes_df = pd.DataFrame({'index': sorted(pd.unique(labs))})
    nodes = hv.Dataset(nodes_df, kdims='index')
    return hv.Chord((links, nodes)).opts(
        opts.Chord(labels='index',
                   node_color='index', cmap='Category20',
                   edge_color='source', edge_cmap='Category20',
                   edge_line_width=dim('value')*2,
                   title=TITLE, width=900, height=900,
                   tools=['hover','tap'])
    )

def chord_by_network_UI(fc, network_labels, default_title="Network-wise FC chord"):
    labs_all = np.asarray(network_labels)
    unique_nets_all = sorted(pd.unique(labs_all))

    w_title    = pn.widgets.TextInput(name="Plot title", value=default_title)
    w_min_abs  = pn.widgets.FloatSlider(name="Min |FC|", start=0.0, end=1.0, step=0.01, value=0.10)
    w_topk_abs = pn.widgets.IntSlider(name="Top-K network pairs (0=all)", start=0, end=500, step=5, value=100)
    w_focus    = pn.widgets.MultiSelect(name="Focus networks (optional)", options=unique_nets_all, size=min(10, len(unique_nets_all)))
    w_order    = pn.widgets.Checkbox(name="Order nodes by total strength", value=True)
    w_hide_iso = pn.widgets.Checkbox(name="Hide isolated networks", value=True)

    @pn.depends(w_title, w_min_abs, w_topk_abs, w_focus, w_order, w_hide_iso)
    def _view(TITLE, min_abs, topk_abs, focus_nets, order_by_strength, hide_isolated):
        # 1) Subset to focused networks (before aggregation)
        if focus_nets:
            keep = np.isin(labs_all, np.array(focus_nets))
            fc_sub = np.asarray(fc)[np.ix_(keep, keep)]
            labs   = labs_all[keep]
        else:
            fc_sub = np.asarray(fc); labs = labs_all

        # 2) Build pair table & aggregate mean |FC| per network pair
        iu = np.triu_indices_from(fc_sub, 1)
        df = pd.DataFrame({'i': iu[0], 'j': iu[1], 'w': fc_sub[iu]})
        df['src'] = labs[df['i']]; df['tgt'] = labs[df['j']]
        df['absw'] = df['w'].abs()
        df = df[df['absw'] >= float(min_abs)]
        if df.empty:
            return pn.pane.Markdown("**No network pairs survive these filters.**\n\n"
                                    "Try lowering _Min |FC|_ or clearing focus.", sizing_mode="stretch_both")

        links_all = (df.groupby(['src','tgt'], as_index=False)['absw']
                       .mean().sort_values('absw', ascending=False)
                       .rename(columns={'src':'source','tgt':'target','absw':'value'}))

        # 3) Sync slider max to available pairs and apply Top-K
        max_pairs = len(links_all)
        if w_topk_abs.end != max_pairs:   # keep UI in sync
            w_topk_abs.end = max_pairs
        k = int(topk_abs)
        links = links_all if (k == 0 or k >= max_pairs) else links_all.head(k)
        links = links[['source','target','value']]

        # 4) Decide which networks (nodes) to show
        engaged = sorted(set(links['source']).union(set(links['target'])))
        if not engaged:
            return pn.pane.Markdown("**No links after Top-K.** Increase _Top-K_ or lower _Min |FC|_.",
                                    sizing_mode="stretch_both")

        if order_by_strength:
            strength = (pd.concat([
                           links.groupby('source')['value'].sum().rename('s'),
                           links.groupby('target')['value'].sum().rename('s')
                       ], axis=0).groupby(level=0).sum()
                       .reindex(engaged, fill_value=0.0))
            node_order = strength.sort_values(ascending=False).index.tolist()
        else:
            node_order = engaged

        if hide_isolated:
            nodes_list = node_order  # only engaged nodes
        else:
            # include all available (focused) networks, but keep engaged first
            rest = [n for n in sorted(pd.unique(labs)) if n not in engaged]
            nodes_list = node_order + rest

        nodes = hv.Dataset(pd.DataFrame({'index': nodes_list}), kdims='index')

        chord = hv.Chord((links, nodes)).opts(
            opts.Chord(labels='index',
                       node_color='index', cmap='Category20',
                       edge_color='source', edge_cmap='Category20',
                       edge_line_width=dim('value')*2,
                       title=TITLE, width=900, height=900,
                       tools=['hover','tap'])
        )

        # 5) small status footer so you can see it reacting
        # footer = pn.pane.Markdown(
        #     f"**Pairs shown:** {len(links)} / {max_pairs}  \n"
        #     f"**Networks shown:** {len(nodes_list)}",
        #     style={'opacity': 0.8}
        # )
        # return pn.Column(chord, footer, sizing_mode="stretch_both")

        footer = pn.pane.Markdown(
            f"**Pairs shown:** {len(links)} / {max_pairs}  \n"
            f"**Networks shown:** {len(nodes_list)}",
            styles={"opacity": "0.8"}   # <-- use styles= (string values are safest)
        )
        return pn.Column(chord, footer, sizing_mode="stretch_both")



    
    controls = pn.Column(
        "### Controls", w_title, w_min_abs, w_topk_abs, w_focus, w_order, w_hide_iso,
        sizing_mode="stretch_width"
    )
    return pn.Row(controls, pn.panel(_view, sizing_mode="stretch_both"))


In [ ]:
#---- launch (new tab) ----
# Your_Corr_matrix : NxN
# Labels           : Nx1
app = chord_by_network_UI(Your_Corr_matrix, Labels,
                          default_title="Network-wise FC Chord")
server = app.show(threaded=True, title="Network Chord | Interactive")